In [1]:
import requests as req;
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException   
from selenium.common.exceptions import TimeoutException, WebDriverException

import time



## Funciones

Esta funcion se usa para extraer los registros de interes de cada pagina (*tipo_vivienda*,*ciudad_vivienda*, *barrio*, *dptos*, *precio*s, *urls*, *numero_habitaciones*, *metros*, *pauta_destacado* y *pauta_oportunidad*),
Inicialmente se recibe el parametro _webpage_, el cual 

In [2]:

def getRegistros(webpage):  
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
    */*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch, br",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "no-cache",
    "dnt": "1",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
    37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
    }
    page = req.get(webpage, headers=headers)
    soup = BeautifulSoup(page.text, 'lxml')
    print("Procesando pagina: ", soup.title.string)
    tipo_vivienda = []
    ciudad_vivienda = []
    barrio = []
    dptos = []
    precios = []
    urls = []
    numero_habitaciones = []
    metros =[]    
    pauta_destacado = []
    pauta_oportunidad = []
    viviendas = pd.DataFrame()
    
    try:    
        ul = soup.find_all('ul', class_="advert")
        for li in ul:
            
            ## Titulo de la publicacion, ciudad y barrio
            title = li.find('div', class_='span-title').h2.text.lower().split('-')
            if len(title) < 2:
                # titulo_registro = title[0].strip()
                tipo_inmueble = title[0].strip().split(' en ')[0]
                ciudad_inmueble = title[0].strip().split(' en ')[1]
                barrio_registro = "sin_informacion"
            else:
                # titulo_registro = title[0].strip()  
                tipo_inmueble = title[0].strip().split(' en ')[0]
                ciudad_inmueble = title[0].strip().split(' en ')[1]
                barrio_registro = title[1].strip()
            
            ## Link Personal a la pagina
            dpto  = li.find('div', class_="span-title").a
            url   = dpto.attrs.get('href')  

            ## Precio
            precio = li.find(attrs={'itemprop':'price'}).get('content')    

            ## Metros y Habitaciones
            info_vivienda = li.find_all(True, {"class":["surface", "li_advert"]})[0].text.strip().lower().split()  
            metros.append(info_vivienda[0])
            
            if len(info_vivienda) > 2:
                numero_habitaciones.append(info_vivienda[2])      
            else:
                numero_habitaciones.append("sin_informacion") 
                
            ## Etiquetas de Oportunidad
            pub_oportunidad = li.find('span', class_='etiqueta')   
            if pub_oportunidad is not None:                
                if len(pub_oportunidad.text) > 0:
                    pauta_oportunidad.append("1")
                else:
                    pauta_oportunidad.append("0")
                
            ## Pautas con la pagina
            pub_destacada = li.find('div', class_='icoov')   
            if pub_destacada is not None:                 
                if pub_destacada:
                    pauta_destacado.append("1")
                else:
                    pauta_destacado.append("0")           
            
            #### Extrayendo departamento
            subt  = dpto.find_all('div')
            indicador_dpto = 1
            for div in subt:
                if indicador_dpto == 2:
                    dpto_title = div.text.strip().lower()            
                indicador_dpto += 1
                
            ## Set Arreglos            
            tipo_vivienda.append(tipo_inmueble)
            ciudad_vivienda.append(ciudad_inmueble)
            barrio.append(barrio_registro)
            dptos.append(dpto_title)
            precios.append(precio)
            urls.append(webpage+url)
            

    
    except AttributeError:
        print("")
        
    ### Seteo del dataframe con los registros
    viviendas = pd.DataFrame()
    viviendas['tipo_vivienda'] = tipo_vivienda
    viviendas['ciudad_vivienda'] = ciudad_vivienda
    viviendas['departamento'] = dptos
    viviendas['barrio'] = barrio
    viviendas['numero_habitaciones'] = numero_habitaciones
    viviendas['metros'] = metros
    viviendas['precio'] = precios
    viviendas['url'] = urls
    viviendas['pauta_destacada'] = pauta_destacado
    viviendas['pauta_oportunidad'] = pauta_oportunidad
    
    
       

    return viviendas
    

La siguiente funcion se utiliza con el fin de conocer si un elemento de la pagina existe, ya que como mecanismo de proteccion, los desarrolladores de la pagina han colocado elementos que cubren la interaccion con elementos de click

In [3]:
def existeEl(idelement):    
    driver.implicitly_wait(5)
#     try:
    try:            
        driver.find_element_by_id(idelement)
        return True
    except WebDriverException as e:
        s = "%s" % e
        print("Got exception %s" % s)
        time.sleep(5)
        return False
#     except NoSuchElementException:
#         return False


### Consulta

### Apertura de url con Selenium

Se hace uso de Selenium debido a que como proceso de prevencion de tecnicas de Scrapping, el portal ha ocultado los links de las paginas a través de funciones *onclick*, donde se evidencia que incialmente los elementos de pagination *a*, no cuentan con id; una vez se ha dado click en el primero y el segundo enlace de la paginacion, estos se transforman en elementos identificables por id.

In [4]:
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)
driver.get('https://www.fincaraiz.com.co/Vivienda/arriendos/')

In [5]:
def getExtraUrls(paginado, urls_registros, cantidad_paginas):
    
    for pag in np.arange(paginado, cantidad_paginas): 
        if existeEl("notification_panel"):        
            driver.execute_script("document.getElementById('notification_panel').remove()")
            driver.implicitly_wait(5)
            try:            
                driver.find_element_by_id("lnkPage" + str(pag)).click()
            except WebDriverException as e:
                s = "%s" % e
                print("Got exception %s" % s)
                time.sleep(5)                   
            urls_registros.append(driver.current_url)  
        else:       
            urls_registros.append(driver.current_url)
            print("Registro numero: ", pag ," guardado sin notificacion ")
            
            try:
                 driver.find_element_by_id("lnkPage" + str(pag)).click()
            except WebDriverException as e:
                s = "%s" % e
                print("Got exception %s" % s)
                time.sleep(5)     
        print("Se ejecuto la pagina: ",pag)  
    
    ## Creacion del dataframe con las urls a scrap
    urls_consulta = pd.DataFrame(urls_registros, columns=['urls'])
    urls_consulta = urls_consulta.drop_duplicates(subset=['urls'])
    return urls_consulta

In [6]:
links = driver.find_elements_by_class_name('link-pag')
urls_registros = []
cantidad_cliks = 1
start = time.time()
print("Empezando proceso 1")

for link in links:   
    print("Procesando pagina: ", cantidad_cliks)
    if cantidad_cliks < 3:
        urls_registros.append(driver.current_url)   
        link.click()               
    else:     
#       funcion para recorrer los elementos por id  
        if existeEl("notification_panel"):        
            driver.execute_script("document.getElementById('notification_panel').remove()")            
            driver.find_element_by_id("lnkPage" + str(cantidad_cliks)).click()       
            urls_registros.append(driver.current_url)  
        else:       
            urls_registros.append(driver.current_url)
            print("Registro numero: ", cantidad_cliks ," guardado sin notificacion ")
            driver.find_element_by_id("lnkPage" + str(cantidad_cliks)).click()
        if cantidad_cliks > 40:
            break
    cantidad_cliks +=1
    
end = time.time()
print("Tiempo transcurrido ",end - start , " Segundos")

Empezando proceso 1
Procesando pagina:  1
Procesando pagina:  2
Procesando pagina:  3
Procesando pagina:  4
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  4  guardado sin notificacion 
Procesando pagina:  5
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  5  guardado sin notificacion 
Procesando pagina:  6
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  6  guardado sin notificacion 
Procesando pagina:  7
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  7  guardado sin notificacion 
Tiempo transcurrido  46.48708438873291  Segundos


### Extra Urls

Debido a la estrucutra del sitio web, se ha separado el proceso de extraccion de las urls

In [ ]:
start = time.time()
print("Empezando proceso 2")
if cantidad_cliks > 7:
    data_urls = getExtraUrls(cantidad_cliks, urls_registros, 2774) #2773
    
end = time.time()
print("Tiempo transcurrido ",end - start)

Empezando proceso 2
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  8  guardado sin notificacion 
Se ejecuto la pagina:  8
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  9  guardado sin notificacion 
Se ejecuto la pagina:  9
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  10  guardado sin notificacion 
Se ejecuto la pagina:  10
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  11  guardado sin notificacion 
Se ejecuto la pagina:  11
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  12  guardado sin notificacion 
Se ejecuto la pagina:  12
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  13  guardado sin notificacion 
Se ejecuto la pagina:  13
Got exception Message: Unable to locate element: [id="notification_panel"]

Re

Registro numero:  62  guardado sin notificacion 
Se ejecuto la pagina:  62
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  63  guardado sin notificacion 
Se ejecuto la pagina:  63
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  64  guardado sin notificacion 
Se ejecuto la pagina:  64
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  65  guardado sin notificacion 
Se ejecuto la pagina:  65
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  66  guardado sin notificacion 
Se ejecuto la pagina:  66
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  67  guardado sin notificacion 
Se ejecuto la pagina:  67
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  68  guardado sin notificacion 
Se ejecuto la pagina:  68
Got exception Messa

Se ejecuto la pagina:  116
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  117  guardado sin notificacion 
Se ejecuto la pagina:  117
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  118  guardado sin notificacion 
Se ejecuto la pagina:  118
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  119  guardado sin notificacion 
Se ejecuto la pagina:  119
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  120  guardado sin notificacion 
Se ejecuto la pagina:  120
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  121  guardado sin notificacion 
Se ejecuto la pagina:  121
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  122  guardado sin notificacion 
Se ejecuto la pagina:  122
Got exception Message: Unable to locate element: [id="n

Registro numero:  170  guardado sin notificacion 
Se ejecuto la pagina:  170
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  171  guardado sin notificacion 
Se ejecuto la pagina:  171
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  172  guardado sin notificacion 
Se ejecuto la pagina:  172
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  173  guardado sin notificacion 
Se ejecuto la pagina:  173
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  174  guardado sin notificacion 
Se ejecuto la pagina:  174
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  175  guardado sin notificacion 
Se ejecuto la pagina:  175
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  176  guardado sin notificacion 
Se ejecuto la pagina:  176
Got e

Registro numero:  224  guardado sin notificacion 
Se ejecuto la pagina:  224
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  225  guardado sin notificacion 
Se ejecuto la pagina:  225
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  226  guardado sin notificacion 
Se ejecuto la pagina:  226
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  227  guardado sin notificacion 
Se ejecuto la pagina:  227
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  228  guardado sin notificacion 
Se ejecuto la pagina:  228
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  229  guardado sin notificacion 
Se ejecuto la pagina:  229
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  230  guardado sin notificacion 
Se ejecuto la pagina:  230
Got e

Registro numero:  278  guardado sin notificacion 
Se ejecuto la pagina:  278
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  279  guardado sin notificacion 
Se ejecuto la pagina:  279
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  280  guardado sin notificacion 
Se ejecuto la pagina:  280
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  281  guardado sin notificacion 
Se ejecuto la pagina:  281
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  282  guardado sin notificacion 
Se ejecuto la pagina:  282
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  283  guardado sin notificacion 
Se ejecuto la pagina:  283
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  284  guardado sin notificacion 
Se ejecuto la pagina:  284
Got e

Registro numero:  332  guardado sin notificacion 
Se ejecuto la pagina:  332
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  333  guardado sin notificacion 
Se ejecuto la pagina:  333
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  334  guardado sin notificacion 
Se ejecuto la pagina:  334
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  335  guardado sin notificacion 
Se ejecuto la pagina:  335
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  336  guardado sin notificacion 
Se ejecuto la pagina:  336
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  337  guardado sin notificacion 
Se ejecuto la pagina:  337
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  338  guardado sin notificacion 
Se ejecuto la pagina:  338
Got e

Registro numero:  386  guardado sin notificacion 
Se ejecuto la pagina:  386
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  387  guardado sin notificacion 
Se ejecuto la pagina:  387
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  388  guardado sin notificacion 
Se ejecuto la pagina:  388
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  389  guardado sin notificacion 
Se ejecuto la pagina:  389
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  390  guardado sin notificacion 
Se ejecuto la pagina:  390
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  391  guardado sin notificacion 
Se ejecuto la pagina:  391
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  392  guardado sin notificacion 
Se ejecuto la pagina:  392
Got e

Registro numero:  440  guardado sin notificacion 
Se ejecuto la pagina:  440
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  441  guardado sin notificacion 
Se ejecuto la pagina:  441
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  442  guardado sin notificacion 
Se ejecuto la pagina:  442
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  443  guardado sin notificacion 
Se ejecuto la pagina:  443
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  444  guardado sin notificacion 
Se ejecuto la pagina:  444
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  445  guardado sin notificacion 
Se ejecuto la pagina:  445
Got exception Message: Unable to locate element: [id="notification_panel"]

Registro numero:  446  guardado sin notificacion 
Se ejecuto la pagina:  446
Got e

Una vez obtenido el dataframe con los enlaces a scrappear, se procede a extraer la informacion

In [ ]:
data=pd.DataFrame()
for url in data_urls.urls:
    data_scra = getRegistros(url)
#     print(data_scra)
    data=pd.concat([data,data_scra],sort=True)    
data = data.reset_index(drop=True)

In [ ]:
data.to_csv('scrapp_fincaraiz2.csv' ,sep=',')

In [ ]:
# headers = {
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
#     */*;q=0.8",
#     "Accept-Encoding": "gzip, deflate, sdch, br",
#     "Accept-Language": "en-US,en;q=0.8",
#     "Cache-Control": "no-cache",
#     "dnt": "1",
#     "Pragma": "no-cache",
#     "Upgrade-Insecure-Requests": "1",
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
#     37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
#     }


# webpage = 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|737||||2||8,9,22,10,24,2||||||||||||||||||||1|||1||griddate%20desc||||||'
# page = req.get(webpage, headers=headers)
# soup = BeautifulSoup(page.text, 'lxml')
# print("Title: ",soup.title.string)
# tipo_vivienda = []
# ciudad_vivienda = []
# barrio = []
# dptos = []
# precios = []
# urls = []
# numero_habitaciones = []
# metros =[]
# pauta_destacado = []
# pauta_oportunidad = []
# viviendas = pd.DataFrame()


# ul = soup.find_all('ul', class_="advert")
# for li in ul:
    
#     ## Titulo de la publicacion, ciudad y barrio
#     title = li.find('div', class_='span-title').h2.text.lower().split('-')
#     if len(title) < 2:
# #         titulo_registro = title[0].strip()
#         tipo_inmueble = title[0].strip().split(' en ')[0]
#         ciudad_inmueble = title[0].strip().split(' en ')[1]
#         barrio_registro = "sin_informacion"
#     else:
# #         titulo_registro = title[0].strip()  
#         tipo_inmueble = title[0].strip().split(' en ')[0]
#         ciudad_inmueble = title[0].strip().split(' en ')[1]
#         barrio_registro = title[1].strip()
    
#     ## Link Personal a la pagina
#     dpto  = li.find('div', class_="span-title").a
#     url   = dpto.attrs.get('href')  
    
#     ## Precio
#     precio = li.find(attrs={'itemprop':'price'}).get('content')    
    
#     ## Metros y Habitaciones
#     info_vivienda = li.find_all(True, {"class":["surface", "li_advert"]})[0].text.strip().lower().split()  
    
#     ## Etiquetas de Oportunidad
#     pub_oportunidad = li.find('span', class_='etiqueta')    
# #     print(pub_oportunidad)
#     if len(pub_oportunidad.text) > 0:
#         pauta_oportunidad.append("1")
#     else:
#         pauta_oportunidad.append("0")
        
#     ## Pautas con la pagina
#     pub_destacada = li.find('div', class_='icoov')    
#     if pub_destacada:
#         pauta_destacado.append("1")
#     else:
#         pauta_destacado.append("0")

#     #### Extrayendo departamento
#     subt  = dpto.find_all('div')
#     indicador_dpto = 1
#     for div in subt:
#         if indicador_dpto == 2:
#             dpto_title = div.text.strip().lower()            
#         indicador_dpto += 1

#     ##### Asiganacion a los arreglos    
#     tipo_vivienda.append(tipo_inmueble)
#     ciudad_vivienda.append(ciudad_inmueble)
#     barrio.append(barrio_registro)
#     dptos.append(dpto_title)
#     precios.append(precio)
#     urls.append(webpage+url)
#     metros.append(info_vivienda[0])
#     if len(info_vivienda[0]) > 2:
#         numero_habitaciones.append(info_vivienda[2])  
#     else:
#         numero_habitaciones.append("sin_informacion")  
        
        
   
        


In [7]:
urls_registros

['https://www.fincaraiz.com.co/Vivienda/arriendos/',
 'https://www.fincaraiz.com.co/Vivienda/arriendos/',
 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|2||||2||8,9,22,10,24,2|||||||||||||||||||||||1||griddate%20desc||||||',
 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|3||||2||8,9,22,10,24,2||||||||||||||||||||1|||1||griddate%20desc||||||',
 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|4||||2||8,9,22,10,24,2||||||||||||||||||||1|||1||griddate%20desc||||||',
 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|5||||2||8,9,22,10,24,2||||||||||||||||||||1|||1||griddate%20desc||||||',
 'https://www.fincaraiz.com.co/Vivienda/arriendo/?ad=30|6||||2||8,9,22,10,24,2||||||||||||||||||||1|||1||griddate%20desc||||||']